In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels
import statsmodels.api as sm
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', None)

In [2]:
# Many columns have NA as values like no basement, looks like pandas is treating them as missing values
# hence read the file with no default na
house_dummy = pd.read_csv("/Users/mannam.sriram/Documents/personal/EXPG/AdvancedRegression/Assignment/train.csv", keep_default_na = False)
house_dummy.shape
house_dummy.head()
house_dummy.info(verbose = True)

(1460, 81)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500
1,2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500
2,3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,9,2008,WD,Normal,223500
3,4,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,12,2008,WD,Normal,250000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             1460 non-null   int64 
 1   MSSubClass     1460 non-null   int64 
 2   MSZoning       1460 non-null   object
 3   LotFrontage    1460 non-null   object
 4   LotArea        1460 non-null   int64 
 5   Street         1460 non-null   object
 6   Alley          1460 non-null   object
 7   LotShape       1460 non-null   object
 8   LandContour    1460 non-null   object
 9   Utilities      1460 non-null   object
 10  LotConfig      1460 non-null   object
 11  LandSlope      1460 non-null   object
 12  Neighborhood   1460 non-null   object
 13  Condition1     1460 non-null   object
 14  Condition2     1460 non-null   object
 15  BldgType       1460 non-null   object
 16  HouseStyle     1460 non-null   object
 17  OverallQual    1460 non-null   int64 
 18  OverallCond    1460 non-null

## Approach
- Data Cleaning
    - Handle missing values
    - Check data mismatched columns
    - Do plot of numerical and categorical columns against output variable and check wheterh linear relationship exists or not
    - Do heap map to check the colinearity among the varibles
    - Drop the columns which are higly correlated
- Covert to dummies
- Train test split (70-30)
- Train Data
    - Normalize numerical columns
    - Build linear MLR with all features the features or using automatice feature selection
        - Drop columns using VIF score and high P-value
        - repeat untill VIF of each columns is < 5
        - Do Residual analysis and check MLR assumptions 
        - If residual analysis matches the assumptions then, transform the numrical columns and predict on train data
        - If assumptions are not true, then check if there is any non linear relationship
        
     - Build MLR with Ridge regularization
     - Build MLR with Laso regularization 

## Data Cleaning

In [3]:
## Correcting column data types
cols_to_be_dropped = ["Id"]
col_to_be_coverted_to_int = ["LotFrontage", "MasVnrArea", ""]
cols_to_be_converted_to_object = ["OverallQual", "OverallCond"]
categorical_cols = ["MSSubClass", "MSZoning","Street", "Alley", "LotShape", "LandContour", 
                    "Utilities", "LotConfig", "LandSlope", "Neighborhood", "Condition1", 
                    "Condition2", "BldgType", "HouseStyle", "OverallQual", "OverallCond", 
                    "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType",
                    "ExterQual", "ExterCond", "Foundation", "BsmtQual", "BsmtCond", "BsmtExposure",
                    "BsmtFinType1", "BsmtFinType2", "Heating", "HeatingQC", "CentralAir", "Electrical",
                    "KitchenQual", "Functional", "FireplaceQu", "GarageType", "GarageQual", "GarageCond",
                    "PavedDrive", "PoolQC", "Fence", "MiscFeature", "SaleType", "SaleCondition",
                    "CentralAir", "GarageFinish"
                    ]

numerical_cols = ["LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF",
                 "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "LowQualFinSF", "GrLivArea",
                 "GarageCars", "GarageArea", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch",
                 "ScreenPorch", "PoolArea", "MiscVal", "YearBuilt", "YearRemodAdd", "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", 
                "BedroomAbvGr", "KitchenAbvGr", "TotRmsAbvGrd", "Fireplaces", "GarageYrBlt", "MoSold",
                "YrSold"]

output_variable = ["SalePrice"]

#Columns that are having NA as a catagory as per the data dictionary
cols_with_na_as_category = ["Alley" , "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2",
                            "FireplaceQu", "GarageType", "GarageFinish", "GarageQual", "GarageCond", "PoolQC",
                            "Fence", "MiscFeature"]

cols_with_none_as_category = ["MasVnrType"]

In [4]:
#as "NA" is a category rename to "NO", so that "NA" is not treated as a null value
new_house = house_dummy.copy()
new_house[cols_with_na_as_category] = house_dummy[cols_with_na_as_category].replace(["NA"], 'NO')
new_house[cols_with_none_as_category] = house_dummy[cols_with_none_as_category].replace(["None"], 'NO')
new_house.drop(cols_to_be_dropped, inplace=True, axis = 1)

#save this new_house to a csv and read that csv with keep_default_na = True
new_house.to_csv("/Users/mannam.sriram/Documents/personal/EXPG/AdvancedRegression/Assignment/train_cleaned.csv", index=False)

In [5]:
house = pd.read_csv("/Users/mannam.sriram/Documents/personal/EXPG/AdvancedRegression/Assignment/train_cleaned.csv")
house.shape
house.head()
house.info(verbose = True)
house.isnull().mean()*100

(1460, 80)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NO,Reg,Lvl,AllPub,Inside,...,0,NO,NO,NO,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NO,Reg,Lvl,AllPub,FR2,...,0,NO,NO,NO,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NO,IR1,Lvl,AllPub,Inside,...,0,NO,NO,NO,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NO,IR1,Lvl,AllPub,Corner,...,0,NO,NO,NO,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NO,IR1,Lvl,AllPub,FR2,...,0,NO,NO,NO,0,12,2008,WD,Normal,250000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

MSSubClass        0.000000
MSZoning          0.000000
LotFrontage      17.739726
LotArea           0.000000
Street            0.000000
Alley             0.000000
LotShape          0.000000
LandContour       0.000000
Utilities         0.000000
LotConfig         0.000000
LandSlope         0.000000
Neighborhood      0.000000
Condition1        0.000000
Condition2        0.000000
BldgType          0.000000
HouseStyle        0.000000
OverallQual       0.000000
OverallCond       0.000000
YearBuilt         0.000000
YearRemodAdd      0.000000
RoofStyle         0.000000
RoofMatl          0.000000
Exterior1st       0.000000
Exterior2nd       0.000000
MasVnrType        0.547945
MasVnrArea        0.547945
ExterQual         0.000000
ExterCond         0.000000
Foundation        0.000000
BsmtQual          0.000000
BsmtCond          0.000000
BsmtExposure      0.000000
BsmtFinType1      0.000000
BsmtFinSF1        0.000000
BsmtFinType2      0.000000
BsmtFinSF2        0.000000
BsmtUnfSF         0.000000
T

In [6]:
#Covering overallcond, overallquality to categorical
# converting symboling to categorical
house[cols_to_be_converted_to_object] = house[cols_to_be_converted_to_object].astype('object')
house.info(verbose=True)
house.isnull().mean()*100

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   object 
 17  OverallCond    1460 non-null   object 
 18  YearBuil

MSSubClass        0.000000
MSZoning          0.000000
LotFrontage      17.739726
LotArea           0.000000
Street            0.000000
Alley             0.000000
LotShape          0.000000
LandContour       0.000000
Utilities         0.000000
LotConfig         0.000000
LandSlope         0.000000
Neighborhood      0.000000
Condition1        0.000000
Condition2        0.000000
BldgType          0.000000
HouseStyle        0.000000
OverallQual       0.000000
OverallCond       0.000000
YearBuilt         0.000000
YearRemodAdd      0.000000
RoofStyle         0.000000
RoofMatl          0.000000
Exterior1st       0.000000
Exterior2nd       0.000000
MasVnrType        0.547945
MasVnrArea        0.547945
ExterQual         0.000000
ExterCond         0.000000
Foundation        0.000000
BsmtQual          0.000000
BsmtCond          0.000000
BsmtExposure      0.000000
BsmtFinType1      0.000000
BsmtFinSF1        0.000000
BsmtFinType2      0.000000
BsmtFinSF2        0.000000
BsmtUnfSF         0.000000
T

In [7]:
#COls having null values, lets impute them
#Impute LotFrontage
simple_imp = SimpleImputer(missing_values=np.nan, strategy='median')
house["LotFrontage"] = simple_imp.fit_transform(house["LotFrontage"].values.reshape(-1, 1))
house["MasVnrArea"] = simple_imp.fit_transform(house["MasVnrArea"].values.reshape(-1, 1))
house["GarageYrBlt"] = simple_imp.fit_transform(house["GarageYrBlt"].values.reshape(-1, 1))

#catagorical
simple_imp_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
house[categorical_cols] = simple_imp_cat.fit_transform(house[categorical_cols])

house.isnull().mean()

MSSubClass       0.0
MSZoning         0.0
LotFrontage      0.0
LotArea          0.0
Street           0.0
Alley            0.0
LotShape         0.0
LandContour      0.0
Utilities        0.0
LotConfig        0.0
LandSlope        0.0
Neighborhood     0.0
Condition1       0.0
Condition2       0.0
BldgType         0.0
HouseStyle       0.0
OverallQual      0.0
OverallCond      0.0
YearBuilt        0.0
YearRemodAdd     0.0
RoofStyle        0.0
RoofMatl         0.0
Exterior1st      0.0
Exterior2nd      0.0
MasVnrType       0.0
MasVnrArea       0.0
ExterQual        0.0
ExterCond        0.0
Foundation       0.0
BsmtQual         0.0
BsmtCond         0.0
BsmtExposure     0.0
BsmtFinType1     0.0
BsmtFinSF1       0.0
BsmtFinType2     0.0
BsmtFinSF2       0.0
BsmtUnfSF        0.0
TotalBsmtSF      0.0
Heating          0.0
HeatingQC        0.0
CentralAir       0.0
Electrical       0.0
1stFlrSF         0.0
2ndFlrSF         0.0
LowQualFinSF     0.0
GrLivArea        0.0
BsmtFullBath     0.0
BsmtHalfBath 

In [8]:
#Column value mapping
# CentralAir has Y and N map to 1 and 0 and make

varlist = ["CentralAir"]

def binary_map(x):
    return x.map({'Y': 1, 'N': 0})

# Applying the function to the housing list
house[varlist] = house[varlist].apply(binary_map)

## Data visualization

In [ ]:
#Heat Map
plt.figure(figsize=(30, 20))
sns.heatmap(house.corr(numeric_only=True), annot = True)
plt.show()

## Convert to dummies

In [9]:
# split into X and y
y = house.pop('SalePrice') # response variable in Y
X = house

In [10]:
pd.set_option('display.max_rows', 500)

house_categorical = X.select_dtypes(include=['object'])
house_categorical.head()

dummies = pd.get_dummies(house_categorical)
X.drop(list(house_categorical.columns), axis = 1, inplace = True)
X = pd.concat([house, dummies], axis = 1)
X.shape

,MSSubClass,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,60,RL,Pave,NO,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,Attchd,RFn,TA,TA,Y,NO,NO,NO,WD,Normal
1,20,RL,Pave,NO,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,Attchd,RFn,TA,TA,Y,NO,NO,NO,WD,Normal
2,60,RL,Pave,NO,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,Attchd,RFn,TA,TA,Y,NO,NO,NO,WD,Normal
3,70,RL,Pave,NO,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,Detchd,Unf,TA,TA,Y,NO,NO,NO,WD,Abnorml
4,60,RL,Pave,NO,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,Attchd,RFn,TA,TA,Y,NO,NO,NO,WD,Normal


(1460, 332)

In [11]:
# split into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.7,
                                                    test_size = 0.3, random_state=100)

In [12]:
house_numerical = X_train.select_dtypes(include=['int64', 'float64'])
house_numerical.columns

Index(['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'CentralAir',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

## Normalize numerical columns using min max scaling on train data

In [13]:
from sklearn.preprocessing import MinMaxScaler

house_numerical = X.select_dtypes(include=['int64', 'float64'])
house_numerical

min_max_scaler = MinMaxScaler()
min_max_scaler.fit(X_train[house_numerical.columns])
X_train[house_numerical.columns] = min_max_scaler.transform(X_train[house_numerical.columns])
X_train[house_numerical.columns].head()

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,CentralAir,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,65.0,8450,2003,2003,196.0,706,0,150,856,1,...,548,0,61,0,0,0,0,0,2,2008
1,80.0,9600,1976,1976,0.0,978,0,284,1262,1,...,460,298,0,0,0,0,0,0,5,2007
2,68.0,11250,2001,2002,162.0,486,0,434,920,1,...,608,0,42,0,0,0,0,0,9,2008
3,60.0,9550,1915,1970,0.0,216,0,540,756,1,...,642,0,35,272,0,0,0,0,2,2006
4,84.0,14260,2000,2000,350.0,655,0,490,1145,1,...,836,192,84,0,0,0,0,0,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,62.0,7917,1999,2000,0.0,0,0,953,953,1,...,460,0,40,0,0,0,0,0,8,2007
1456,85.0,13175,1978,1988,119.0,790,163,589,1542,1,...,500,349,0,0,0,0,0,0,2,2010
1457,66.0,9042,1941,2006,0.0,275,0,877,1152,1,...,252,0,60,0,0,0,0,2500,5,2010
1458,68.0,9717,1950,1996,0.0,49,1029,0,1078,1,...,240,366,0,112,0,0,0,0,4,2010


MinMaxScaler()

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,CentralAir,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
210,0.157534,0.019306,0.384058,0.000000,0.00,0.082920,0.0,0.169521,0.141408,0.0,...,0.000000,0.000000,0.000000,0.173913,0.0,0.0,0.0,0.0,0.272727,0.50
318,0.236301,0.039403,0.876812,0.716667,0.16,0.174876,0.0,0.154110,0.220458,1.0,...,0.462623,0.396733,0.114723,0.260870,0.0,0.0,0.0,0.0,0.272727,0.75
239,0.106164,0.033981,0.528986,0.000000,0.00,0.016655,0.0,0.274401,0.120295,1.0,...,0.155148,0.000000,0.267686,0.000000,0.0,0.0,0.0,0.0,0.272727,1.00
986,0.130137,0.017931,0.275362,0.883333,0.00,0.000000,0.0,0.207620,0.079378,1.0,...,0.179831,0.459743,0.000000,0.000000,0.0,0.0,0.0,0.0,0.454545,0.00
1416,0.133562,0.046139,0.094203,0.000000,0.00,0.000000,0.0,0.332620,0.127169,1.0,...,0.394922,0.000000,0.000000,0.206522,0.0,0.0,0.0,0.0,0.272727,1.00


In [14]:
## scale test data
X_test[house_numerical.columns] = min_max_scaler.transform(X_test[house_numerical.columns])
X_test[house_numerical.columns].head()

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,CentralAir,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
1436,0.133562,0.035192,0.717391,0.350000,0.00000,0.109142,0.0,0.106164,0.141408,1.0,...,0.372355,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.363636,0.25
57,0.232877,0.047566,0.956522,0.900000,0.00000,0.000000,0.0,0.368151,0.140753,1.0,...,0.398449,0.000000,0.133843,0.0,0.0,0.0,0.000000,0.0,0.636364,0.00
780,0.143836,0.029930,0.891304,0.766667,0.02375,0.000000,0.0,0.529538,0.202455,1.0,...,0.283498,0.256709,0.040153,0.0,0.0,0.0,0.000000,0.0,0.454545,0.25
382,0.198630,0.036338,0.971014,0.933333,0.00000,0.000000,0.0,0.401969,0.153682,1.0,...,0.450635,0.168028,0.101338,0.0,0.0,0.0,0.000000,0.0,0.272727,0.25
1170,0.188356,0.039309,0.760870,0.450000,0.00000,0.092488,0.0,0.245719,0.179378,1.0,...,0.252468,0.236873,0.000000,0.0,0.0,0.0,0.888889,0.0,0.545455,0.50


## MLR with all the features

In [15]:
def cal_scores(y_train_actual, y_hat_train, y_test_actual, y_hat_test):
    metric = []
    r2_train_lr = r2_score(y_train_actual, y_hat_train)
    print(f'r2_train_lr {r2_train_lr}')
    metric.append(r2_train_lr)

    r2_test_lr = r2_score(y_test_actual, y_hat_test)
    print(f'r2_test_lr {r2_test_lr}')
    metric.append(r2_test_lr)

    rss1_lr = np.sum(np.square(y_train_actual - y_hat_train))
    print(f'rss1_lr {rss1_lr}')
    metric.append(rss1_lr)

    rss2_lr = np.sum(np.square(y_test_actual - y_hat_test))
    print(f'rss2_lr {rss2_lr}')
    metric.append(rss2_lr)

    mse_train_lr = mean_squared_error(y_train_actual, y_hat_train)
    print(f'mse_train_lr {mse_train_lr}')
    metric.append(mse_train_lr**0.5)

    mse_test_lr = mean_squared_error(y_test_actual, y_hat_test)
    print(f'mse_test_lr {mse_test_lr}')
    metric.append(mse_test_lr**0.5)
    
    return metric

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)
print(lm.intercept_)
print(lm.coef_)

y_pred_train = lm.predict(X_train)
y_pred_test = lm.predict(X_test)

mlr_all_featues_scores = cal_scores(y_train, y_pred_train, y_test, y_pred_test)

## MLR with RFE

In [16]:
def calculate_vif_score(df):
    vif = pd.DataFrame()
    vif['Features'] = df.columns
    vif['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif

In [ ]:
## Linear Regerssion with RFE
lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, n_features_to_select = 20)             # running RFE
rfe = rfe.fit(X_train, y_train)
#list(zip(X_train.columns,rfe.support_,rfe.ranking_))

#Building model using statsmodel, for the detailed statistics
X_train_rfe = X_train[X_train.columns[rfe.support_]]
lr = lm.fit(X_train_rfe, y_train)

calculate_vif_score(X_train_rfe.astype(float))

#Making predictions on the train data
y_train_pred = lr.predict(X_train_rfe)
train_r2_score = r2_score(y_train, y_train_pred)

# Plot the histogram of the error terms
fig = plt.figure()
sns.histplot((y_train - y_train_pred), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)
plt.xlabel('Errors', fontsize = 18) 
plt.show()

#Scatter plot of the error terms
sns.scatterplot(y_train - y_train_pred)
plt.show()

## Making predictions on test data
X_test_rfe = X_test[X_test.columns[rfe.support_]]
y_test_pred_rfe = lr.predict(X_test_rfe)
test_r2_score = r2_score(y_test, y_test_pred_rfe)

mlr_with_rfe_scores = cal_scores(y_train, y_train_pred, y_test, y_test_pred_rfe)

In [17]:
## using Regularization

# list of alphas to tune - if value too high it will lead to underfitting, if it is too low, 
# it will not handle the overfitting
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20.0, 50.0, 100.0, 500.0, 1000.0 ]}

In [19]:
ridge = Ridge()

# cross validation
folds = 5
model_cv = GridSearchCV(estimator = ridge, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error',  
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            
model_cv.fit(X_train, y_train) 
#https://scikit-learn.org/stable/modules/model_evaluation.html

best_alpha_ridge = model_cv.best_params_["alpha"]
print(f'best_alpha {best_alpha_ridge}')

#Fitting Ridge model for alpha = 10 and printing coefficients which have been penalised
ridge = Ridge(alpha=best_alpha_ridge)

ridge.fit(X_train, y_train)
print(ridge.coef_)

# Lets calculate some metrics such as R2 score, RSS and RMSE
y_pred_train_ridge = ridge.predict(X_train)
y_pred_test_ridge = ridge.predict(X_test)

ridge_scores = cal_scores(y_train, y_pred_train_ridge, y_test, y_pred_test_ridge)

Fitting 5 folds for each of 28 candidates, totalling 140 fits


GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3,
                                   0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0,
                                   4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20.0,
                                   50.0, 100.0, 500.0, 1000.0]},
             return_train_score=True, scoring='neg_mean_absolute_error',
             verbose=1)

best_alpha 0.2


Ridge(alpha=0.2)

[ 1.06698279e+04  8.82883412e+04  4.58823789e+04  5.71996283e+03
  2.59116217e+04  8.30040810e+04  2.28332902e+04  1.09870850e+04
  8.63825096e+04  1.10855953e+03  1.09180866e+05  8.89125031e+04
 -1.77700029e+04  1.24157208e+05  1.18672138e+04 -1.78615415e+03
  1.14924500e+04 -2.19832539e+03 -1.02249033e+04 -2.93015804e+04
  2.97585993e+03  8.18084623e+03  1.72299913e+03  2.40118784e+04
  1.23049284e+04  6.49021264e+03  1.11287767e+04  3.66701097e+03
  2.19198459e+04  8.96096311e+03  6.99860508e+03  8.42817243e+03
 -3.82426500e+03 -1.74814216e+03  5.73149586e+03  5.61543266e+03
  5.64778339e+02 -4.10003453e+03  3.48818150e+03  7.49659387e+03
  8.77082312e+03 -2.26777297e+04  2.60701947e+03 -2.75475604e+03
 -5.12103997e+02  6.56386482e+00  3.84601145e+02 -2.06201895e+03
 -2.55884663e+03 -2.98005731e+04  6.15885119e+03  9.23607982e+03
  8.66315905e+03  5.74248304e+03 -7.61672122e+03  7.61672122e+03
 -4.26393993e+02 -3.80335249e+02  8.06729242e+02 -2.51653907e+03
  1.39146481e+03  2.69521

In [20]:
#Most imp predictor 
# with ridge best alpha is 0.2, double it and run with 0.4
ridge = Ridge(alpha=best_alpha_ridge * 2)

ridge.fit(X_train, y_train)
print(ridge.coef_)

# Lets calculate some metrics such as R2 score, RSS and RMSE
y_pred_train_ridge = ridge.predict(X_train)
y_pred_test_ridge = ridge.predict(X_test)

ridge_scores = cal_scores(y_train, y_pred_train_ridge, y_test, y_pred_test_ridge)

Ridge(alpha=0.4)

[ 3.68855007e+03  6.75017001e+04  3.75451555e+04  6.03219641e+03
  2.54554956e+04  6.19420638e+04  1.95328860e+04  1.05775904e+04
  6.59740971e+04  2.02098289e+03  9.68111429e+04  8.27519270e+04
 -1.64082949e+04  1.11578900e+05  1.49749938e+04 -9.98734595e+02
  1.58840275e+04 -1.09750022e+02 -7.11630322e+03 -2.60684521e+04
  1.00172686e+04  9.47198481e+03  2.72209487e+01  2.76899453e+04
  1.14911991e+04  7.72985057e+03  9.34794624e+03  3.37066565e+03
  2.27104130e+04  9.26251223e+03  3.32246286e+03  4.60142997e+03
 -3.63648694e+03 -1.86407616e+03  5.75703683e+03  3.84672222e+03
  1.54639766e+03 -3.75442689e+03  3.66999057e+03  5.71325487e+03
  7.99511890e+03 -1.61468664e+04  2.78062611e+03 -2.11333129e+03
 -3.56598591e+02 -1.50488905e+03 -1.18701673e+03 -3.67778847e+03
 -2.56822968e+03 -2.85926489e+04  5.66169019e+03  8.80794998e+03
  8.21119221e+03  5.91181652e+03 -5.53142496e+03  5.53142496e+03
 -1.69098874e+02 -5.31566226e+02  7.00665101e+02 -2.46014334e+03
  1.98060861e+03  1.95233

## Lasso

In [70]:
lasso = Lasso()

# cross validation
model_lasso = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            

model_lasso.fit(X_train, y_train)

# Printing the best hyperparameter alpha
best_alpha_lasso = model_lasso.best_params_["alpha"]
print(f'best_alpha {best_alpha_lasso}')

#Fitting Ridge model for alpha = 100 and printing coefficients which have been penalised

lasso = Lasso(alpha=best_alpha_lasso)
lasso.fit(X_train, y_train)
lasso.coef_

# Lets calculate some metrics such as R2 score, RSS and RMSE

y_pred_train_lasso = lasso.predict(X_train)
y_pred_test_lasso = lasso.predict(X_test)
lasso_scores = cal_scores(y_train, y_pred_train_lasso, y_test, y_pred_test_lasso)

Fitting 5 folds for each of 28 candidates, totalling 140 fits


/Users/mannam.sriram/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.998e+10, tolerance: 5.301e+08
  model = cd_fast.enet_coordinate_descent(
/Users/mannam.sriram/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.555e+10, tolerance: 4.851e+08
  model = cd_fast.enet_coordinate_descent(
/Users/mannam.sriram/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

/Users/mannam.sriram/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.582e+10, tolerance: 5.312e+08
  model = cd_fast.enet_coordinate_descent(
/Users/mannam.sriram/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.251e+10, tolerance: 4.884e+08
  model = cd_fast.enet_coordinate_descent(
/Users/mannam.sriram/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

/Users/mannam.sriram/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.711e+09, tolerance: 4.851e+08
  model = cd_fast.enet_coordinate_descent(
/Users/mannam.sriram/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.927e+10, tolerance: 5.169e+08
  model = cd_fast.enet_coordinate_descent(
/Users/mannam.sriram/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

/Users/mannam.sriram/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.125e+10, tolerance: 5.301e+08
  model = cd_fast.enet_coordinate_descent(
/Users/mannam.sriram/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.864e+09, tolerance: 5.312e+08
  model = cd_fast.enet_coordinate_descent(
/Users/mannam.sriram/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3,
                                   0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0,
                                   4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20.0,
                                   50.0, 100.0, 500.0, 1000.0]},
             return_train_score=True, scoring='neg_mean_absolute_error',
             verbose=1)

best_alpha 50.0


Lasso(alpha=50.0)

array([ 0.00000000e+00,  7.35923822e+04,  3.69098057e+04,  8.67512226e+03,
        2.85366354e+04,  1.23308557e+05,  1.94809348e+04,  2.81692403e+04,
        2.05323866e+03,  2.46583971e+05,  1.32611312e+05, -0.00000000e+00,
        1.15034473e+04, -0.00000000e+00,  1.22733728e+04, -0.00000000e+00,
       -7.20782473e+03, -2.51034335e+04, -0.00000000e+00,  7.06296185e+03,
        0.00000000e+00,  2.63775701e+04,  4.85561912e+03,  7.20574405e+03,
        6.65817371e+03, -0.00000000e+00,  9.31115097e+02,  3.36625400e+03,
        0.00000000e+00,  0.00000000e+00, -1.59149935e+03, -1.19724910e+03,
        2.03517435e+03, -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -1.21950424e+04,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -2.84936386e+04,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -3.30342203e+03,
       -0.00000000e+00,  

r2_train_lr 0.9420285204905677
r2_test_lr 0.837493519374845
rss1_lr 369898549709.744
rss2_lr 458059053201.25586
mse_train_lr 362290450.25440156
mse_test_lr 1045796925.1170225


In [27]:
type(lasso.coef_)
max(lasso.coef_)

numpy.ndarray

267614.41108208365

In [71]:
def top_columns(coeff, top_k = 5):
    ind = np.argpartition(coeff, -top_k)[-top_k:]
    top_values = coeff[ind]
    print(top_values)
    top_col = []
    for i in top_values:
        a = np.array(X_train.columns)[coeff == i]
        top_col.extend(a)

    return top_col 

In [73]:
top_lasso_col = top_columns(lasso.coef_, 5)
top_lasso_col
top_columns(ridge.coef_, 5)

[ 73592.38215196 123308.55715677 246583.97128335  84743.36454379
 132611.31200195]


['LotArea', 'BsmtFinSF1', '1stFlrSF', 'PoolQC_Ex', '2ndFlrSF']

[ 82751.92702846  84423.36587526  96811.1428634  115127.70134151
 111578.89981012]


IndexError: boolean index did not match indexed array along dimension 0; dimension is 327 but corresponding boolean dimension is 332

In [69]:
#Remove top 5 lasso columns and re run it
X_train.shape

(1021, 327)

In [68]:
X_train_lasso_drop_top_5 = X_train.drop(top_lasso_col, inplace = True, axis = 1)
X_test_lasso_drop_top_5 = X_test.drop(top_lasso_col, inplace = True, axis = 1)
X_train_lasso_drop_top_5.shape
X_test_lasso_drop_top_5.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [25]:
#double it and test
lasso = Lasso(alpha=best_alpha_lasso*2)
lasso.fit(X_train, y_train)
lasso.coef_

# Lets calculate some metrics such as R2 score, RSS and RMSE

y_pred_train_lasso = lasso.predict(X_train)
y_pred_test_lasso = lasso.predict(X_test)
lasso_scores = cal_scores(y_train, y_pred_train_lasso, y_test, y_pred_test_lasso)

Lasso(alpha=100.0)

array([ 0.00000000e+00,  3.16502990e+04,  2.61117270e+04,  9.70880219e+03,
        1.36294378e+04,  1.61991916e+04,  0.00000000e+00, -0.00000000e+00,
        5.19234663e+04,  1.73587420e+03,  0.00000000e+00,  9.13453378e+03,
       -1.49532662e+04,  2.67614411e+05,  1.70609895e+04,  0.00000000e+00,
        1.05776203e+04,  0.00000000e+00, -0.00000000e+00, -1.22773071e+04,
        0.00000000e+00,  1.04012315e+04,  0.00000000e+00,  2.58029290e+04,
        4.98638947e+03,  4.00672545e+03,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00,  3.52205191e+03,  0.00000000e+00,  0.00000000e+00,
       -1.20436139e+03, -4.42559946e+02,  2.69225044e+03, -0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  2.39927105e+03,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        0.00000000e+00, -1.74763727e+04,  0.00000000e+00,  0.00000000e+00,
        1.72394511e+03, -

r2_train_lr 0.935044428058271
r2_test_lr 0.856603857264153
rss1_lr 414461939907.9363
rss2_lr 404192504333.4349
mse_train_lr 405937257.5004273
mse_test_lr 922813936.8343261


In [ ]:
X_train_lasso_sel_ = X_train.iloc[:,lasso.coef_!=0].astype('float64')
X_test_lasso_sel_ = X_test.iloc[:,lasso.coef_!=0].astype('float64')
calculate_vif_score(X_train_lasso_sel_)

In [ ]:
# SaleCondition_Partial is highly correlaed with some other columns or a combination of other columns
# lets remove it and try it
X_train_lasso_sel_.drop(["SaleCondition_Partial"], inplace = True, axis = 1)
X_test_lasso_sel_.drop(["SaleCondition_Partial"], inplace = True, axis = 1)
X_train_lasso_sel_.shape
X_test_lasso_sel_.shape

In [ ]:
lasso = Lasso()

# cross validation
model_lasso = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            

model_lasso.fit(X_train_lasso_sel_, y_train)

# Printing the best hyperparameter alpha
best_alpha = model_lasso.best_params_["alpha"]
print(f'best_alpha {best_alpha}')

#Fitting Ridge model for alpha = 100 and printing coefficients which have been penalised
lasso = Lasso(alpha=best_alpha)
lasso.fit(X_train_lasso_sel_, y_train)
lasso.coef_

# Lets calculate some metrics such as R2 score, RSS and RMSE

y_pred_train_lasso = lasso.predict(X_train_lasso_sel_)
y_pred_test_lasso = lasso.predict(X_test_lasso_sel_)
lasso_scores = cal_scores(y_train, y_pred_train_lasso, y_test, y_pred_test_lasso)

In [ ]:
X_train_lasso_sel1 = X_train_lasso_sel_.iloc[:,lasso.coef_!=0].astype('float64')
X_test_lasso_sel1 = X_test_lasso_sel_.iloc[:,lasso.coef_!=0].astype('float64')
calculate_vif_score(X_train_lasso_sel1)

In [ ]:
# PoolQC_Gd is highly correlaed with some other columns or a combination of other columns
# lets remove it and try it
X_train_lasso_sel1.drop(["PoolQC_Gd"], inplace = True, axis = 1)
X_test_lasso_sel1.drop(["PoolQC_Gd"], inplace = True, axis = 1)
X_train_lasso_sel1.shape
X_test_lasso_sel1.shape

In [ ]:
lasso = Lasso()

# cross validation
model_lasso = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            

model_lasso.fit(X_train_lasso_sel1, y_train)

# Printing the best hyperparameter alpha
best_alpha = model_lasso.best_params_["alpha"]
print(f'best_alpha {best_alpha}')

#Fitting Ridge model for alpha = 100 and printing coefficients which have been penalised
lasso = Lasso(alpha=best_alpha)
lasso.fit(X_train_lasso_sel1, y_train)
lasso.coef_

# Lets calculate some metrics such as R2 score, RSS and RMSE

y_pred_train_lasso = lasso.predict(X_train_lasso_sel1)
y_pred_test_lasso = lasso.predict(X_test_lasso_sel1)
lasso_scores = cal_scores(y_train, y_pred_train_lasso, y_test, y_pred_test_lasso)

In [ ]:
X_train_lasso_sel2 = X_train_lasso_sel1.iloc[:,lasso.coef_!=0].astype('float64')
X_test_lasso_sel2 = X_train_lasso_sel1.iloc[:,lasso.coef_!=0].astype('float64')
calculate_vif_score(X_train_lasso_sel2)

In [ ]:
#Build a stats model to check the significance of each predictor and remove them 
lr_dec = sm.OLS(y_train, sm.add_constant(X_train_lasso_sel2)).fit()
lr_dec.summary()


In [ ]:
X_train_lasso_sel2["Neighborhood_SawyerW"].unique()

In [ ]:
#Neighborhood_SawyerW has high p-value of 1, drop it
cols_to_be_dropped.append("Neighborhood_SawyerW")
X_train_p = sm.add_constant(X_train_lasso_sel2.drop(["Neighborhood_SawyerW"], axis = 1))

lr_dec = sm.OLS(y_train, X_train_p).fit()
lr_dec.summary()
calculate_vif_score(X_train_p)

In [ ]:
X_train_lasso_sel2.reset_index(drop = True).head()

In [ ]:
#Neighborhood_SawyerW has high p-value of 1, drop it
cols_to_be_dropped.append("BsmtFinType1_Unf")
X_train_p = sm.add_constant(X_train_lasso_sel2.drop(cols_to_be_dropped, axis = 1))

lr_dec = sm.OLS(y_train, X_train_p).fit()
lr_dec.summary()
calculate_vif_score(X_train_p)

In [ ]:
# Creating a table which contain all the metrics

lr_table = {'Metric': ['R2 Score (Train)','R2 Score (Test)','RSS (Train)','RSS (Test)',
                       'MSE (Train)','MSE (Test)'], 
        'Linear Regression': mlr_all_featues_scores
        }

lr_metric = pd.DataFrame(lr_table ,columns = ['Metric', 'Linear Regression'] )
lr_rfe_metric = pd.Series(mlr_with_rfe_scores, name = 'LR with RFE')

rg_metric = pd.Series(ridge_scores, name = 'Ridge Regression')
ls_metric = pd.Series(lasso_scores, name = 'Lasso Regression')

final_metric = pd.concat([lr_metric, lr_rfe_metric, rg_metric, ls_metric], axis = 1)

final_metric